In [1]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import pandas as pd
import copy
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from scipy import stats
from sklearn.metrics import confusion_matrix
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import mutual_info_classif

In [2]:
#加载数据 预处理
data = pd.read_csv("../data/gene_version2020/generate/ExperimentalDatasets/MM/MM-BPCCMF.csv",index_col=0)
label = data["longevity influence"]
#删除无用列
data = data.drop(["longevity influence"],axis=1)

#初始化基因本体的DAG
DAG = pd.read_csv("../data/gene_version2020/generate/GOPath/MM/MM-BPCCMF.csv",index_col=0)

#过滤低纬度特征
for c in data.columns:
    data[c] = data[c].astype('int')
    if(data[c].sum() < 3):
        data.drop(c,axis=1,inplace=True)
DAG = DAG.loc[data.columns,data.columns]

#计算相关性
IGWeight = mutual_info_classif(data,label,discrete_features=True)
IGWeight = pd.Series(IGWeight,index=DAG.columns)
data.shape

(130, 503)

In [3]:
def GM_score(x,y): #计算GM 传入numpy
    cm1 = confusion_matrix(x,y)
    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    return sensitivity1,specificity1

def getPathsToRoot(path,name):#传入list 方便递归
    global pathList
    flag = 0
    path.append(name)
    for anc in DAG.columns:
        if(DAG.loc[anc,name] == 1):
            getPathsToRoot(copy.copy(path),anc)
            flag = 1
    if(flag == 0):
        pathList.append(path)
        
def SHSEL(trainData):
    global pathList
    #找到所有叶子结点
    leafNodes = []
    dropList = []
    returnList = []
    for go in DAG.columns:
        if(DAG.loc[go,:].sum() == 0):
            leafNodes.append(go)
    #step1
    for leaf in leafNodes:
        for node in DAG.columns:
            if(DAG.loc[node,leaf] == 1):
                if((1-abs(IGWeight[node]-IGWeight[leaf])) >= threshold):
                    dropList.append(leaf)
                leafNodes.append(node)
    selectFeatures = list(set(DAG.columns)-set(dropList))
    print(len(selectFeatures))
    #step2
    leafNodes = []
    for go in DAG.columns:
        if(DAG.loc[go,:].sum() == 0):
            leafNodes.append(go)
    for leaf in leafNodes:
        pathList = []
        getPathsToRoot([],leaf)
        for path in pathList:
            path = list(filter(lambda x:x in selectFeatures,path))
            pathIG = IGWeight[path]
            highIG = pathIG[pathIG >= pathIG.mean()]
            for index in highIG.index:
                returnList.append(index)
    return list(set(returnList))

def myknn(train,label,x_test):
    predictList = []
    for j in range(x_test.shape[0]):
        test = x_test.iloc[j,:]
        maxJcd = 0
        prediction = 0
        for i in range(train.shape[0]):
            temp = jaccard_score(train.iloc[i,:],test)
            if(temp > maxJcd):
                prediction = label[i]
                maxJcd = temp
        predictList.append(prediction)
    return predictList

In [4]:
pathList = []
sensitivity = []
specificity = []
F1 = []
AUC = []
threshold = 0.999
#10折交叉验证
kf = KFold(n_splits=10,shuffle=False)
for train_index ,test_index in kf.split(data):
    trainData = data.iloc[train_index,:]
    testData = data.iloc[test_index,:]
    Y_train = label.values[train_index] #用于训练模型
    Y_test = label.values[test_index]#用于交叉验证
    
    selectFeatures = SHSEL(trainData) 
    print(len(selectFeatures))
    X_train = trainData.loc[:,selectFeatures]
    X_test = testData.loc[:,selectFeatures]
    gnb = GaussianNB()
    predictList = gnb.fit(X_train, Y_train).predict(X_test)
    #predictList = myknn(X_train,Y_train,X_test)
    
    print(predictList)
    print(Y_test)
    sensi,speci= GM_score(predictList,Y_test)
    sensitivity.append(sensi)
    specificity.append(speci)
    F1.append(f1_score(np.array(predictList),Y_test))
    AUC.append(roc_auc_score(np.array(predictList),Y_test))

458
346
[0 0 1 1 1 1 0 1 1 1 1 1 1]
[0 1 1 1 1 1 1 1 1 1 1 0 1]
458
346
[1 1 1 1 1 1 1 0 0 0 1 1 0]
[0 1 1 1 1 1 1 0 0 0 0 1 1]
458
346
[1 0 0 0 1 1 1 1 0 1 1 1 1]
[0 0 0 0 1 1 1 1 1 1 1 0 1]
458
346
[1 0 1 1 1 1 1 1 0 1 0 1 0]
[1 0 1 1 1 1 0 1 0 1 1 1 0]
458
346
[0 1 1 1 1 0 1 1 1 1 1 1 1]
[0 1 1 1 1 1 1 1 1 1 1 0 1]
458
346
[1 1 1 1 1 1 0 1 1 1 1 1 0]
[1 1 1 0 0 1 1 0 1 0 1 1 0]
458
346
[1 1 1 1 1 1 1 1 0 0 0 1 1]
[1 1 1 1 0 1 1 1 0 0 0 1 0]
458
346
[0 1 1 1 1 1 1 1 1 0 0 1 1]
[0 0 1 1 1 1 1 1 0 0 0 1 1]
458
346
[1 1 0 1 1 0 1 0 1 0 1 0 0]
[1 1 0 1 0 1 1 1 0 0 0 1 0]
458
346
[1 0 1 1 1 0 1 0 0 1 1 1 0]
[1 0 1 1 0 1 1 0 0 0 1 1 0]


In [6]:
a = np.nanmean(sensitivity)
b = np.nanmean(specificity)
print(a)
print(b)
print("GM")
print(math.sqrt(a*b))
print("f1")
print(np.nanmean(F1))
print("AUC")
print(np.nanmean(AUC))

0.6883333333333332
0.7811327561327561
GM
0.7332664684851706
f1
0.8188229933121575
AUC
0.7347330447330447
